<a href="https://colab.research.google.com/github/TanJiaTing/AIP/blob/master/cdQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Closed-Domain Question Answering using BERT**


In [1]:
# References: https://github.com/cdqa-suite/cdQA
!pip install cdqa
import os
import pandas as pd
from ast import literal_eval
import tensorflow as tf
from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

C:\Users\tanjt\anaconda3\envs\cdqa\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Download pre-trained reader model and PDF files

In [2]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')


bert_qa.joblib already downloaded


In [3]:
# Download AIP pdf
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    model_url = 'https://www.caas.gov.sg/docs/default-source/pdf/aip-singapore---21-may-20.pdf'
    print('\nDownloading PDF file...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    wget.download(url=model_url, out=directory)

download_pdf()

### Convert the PDF files into a DataFrame for cdQA pipeline

In [4]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

,title,paragraphs
0,aip-singapore---21-may-20 (1),"[AMDTeAIPContactPost:, AERONAUTICALINFORMATION..."
1,aip-singapore---21-may-20,"[AMDTeAIPContactPost:, AERONAUTICALINFORMATION..."


### Instantiate the cdQA pipeline from a pre-trained reader model

In [5]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [6]:
# Fine-tuning based on SQUAD-like corpus
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib') # use 'distilbert_qa.joblib' for DistilBERT instead of BERT
cdqa_pipeline.fit_reader('C:/Users/tanjt/Desktop/AIP/data/trainqa.json')

RuntimeError: CUDA out of memory. Tried to allocate 54.00 MiB (GPU 0; 4.00 GiB total capacity; 1019.82 MiB already allocated; 0 bytes free; 90.18 MiB cached)

In [ ]:
#Save model after fine-tuning
cdqa_pipeline.dump_reader('saved.joblib')

 ### Execute a query

In [ ]:
new_pipeline = QAPipeline(reader='./saved.joblib', max_df=1.0)
new_pipeline.fit_retriever(df=df)
query = 'Which agent should be engaged by business aviation flights at Changi Airport?'
prediction = new_pipeline.predict(query)

### Explore predictions

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

In [ ]:
#Evaluate model
from cdqa.utils.evaluation import evaluate_reader

evaluate_reader(new_pipeline, '/content/data/testqa.json')

In [ ]:
#Check gpu details
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()
%ls -l